In [34]:
#"To run this code, either take out all the calls to tqdm or run !pip install tqdm"

from tqdm import tqdm
from copy import copy

In [35]:
#This code does all the work of producing the Specht matroid from the combinatorics of words.

def distinctColumns(w1, w2):
    if len(w1) != len(w2): 
        return False
    seen = set()
    for i in range(len(w1)):
        t = (w1[i], w2[i])
        if t in seen: 
            return False
        seen.add(t)
    return True

def YoungCharacter(w1, w2):
    assert distinctColumns(w1, w2)
    wp = [(w1[i], w2[i]) for i in range(len(w1))]
    def ycfunc(r1, r2):
        if not distinctColumns(r1, r2):
            return 0
        rp = [(r1[i], r2[i]) for i in range(len(w1))]
        po = [wp.index(rx) + 1 for rx in rp]
        return Permutation(po).sign()
    return ycfunc

def SpechtMatrix(w1, w2):
    Y = YoungCharacter(w1, w2)
    rows = [[Y(u, v) for u in Permutations(w1)] for v in Permutations(w2)]
        
    return matrix(QQ, rows).transpose()

def hook_shape(n, k):
    return Partition([n-k] + [1]*k)

def hook_words(n,k):
    w1, w2 = zip(*(hook_shape(n,k).cells()))
    return w1, w2

def hook_matrix(n,k):
    return SpechtMatrix(*hook_words(n,k))

def hook_specht_matroid_slow(n,k):
    M = SpechtMatrix(*hook_words(n,k))
    return Matroid(M)

In [36]:
#This code goes through the matroid constructed above and throws away the negations of any vector. 
#Doing this shrinks the size of the matrix considerably, making later computations easier.
def NarrowerSpechtMatrix(w1, w2):
    Y = YoungCharacter(w1, w2)
    cols = []
    orig_col_num = 0
    colConversion = []
    for v in tqdm(Permutations(w2)):
        new_col = vector([Y(u, v) for u in Permutations(w1)])
        if new_col not in cols and -new_col not in cols: # slow but whatever. future things are slower
            cols.append(new_col)
            colConversion.append(orig_col_num)
        orig_col_num += 1
    return (matrix(QQ, cols).transpose(),colConversion)

In [37]:
#Code that constructs the matroids of hooks. Specializes the above.

def hook_specht_matrix(n,k):
    M,colConversion = NarrowerSpechtMatrix(*hook_words(n,k))
    #rref = M.rref()
    #rowcount = binomial(n-1, k)
    return (M,colConversion) #Matrix(rowcount, M.dimensions()[1], lambda i,j:rref[i,j])
   
def hook_specht_matroid(n,k):
    return Matroid(hook_specht_matrix(n,k)[0])

In [38]:
#The following code will take in a given hook matroid with two boxes below the first row,
#and compute all of its codim 1 flats. It does this by starting with all of the hyperplanes of the matroid, and removing
#them one-by-one until the dimension goes down by 1. It then repeats by selecting different hyperplanes to remove.

def max_rank_flats(n):
    target_rank = binomial(n-1,2)-1
    M = hook_specht_matroid(n,2)
    ground = Set(M.groundset()) 
    groundsize = len(ground)
    work_stack = []
    bigflats = []
    for S in ground.subsets(groundsize-1):
        assert M.rank(S) == target_rank + 1
        work_stack.append(S)   
    step = 0
    while len(work_stack) > 0:
        step += 1
        if step % 100000 == 0:
            print("step", step, " - Stack size now:", len(work_stack), "flats found:", len(bigflats))
        work = work_stack.pop()
        biggest_absent_thing = max(ground-work)
        for t in range(biggest_absent_thing+1, groundsize):
            assert t in work # should work unless I don't know what "Biggest" means
            S = work.difference(Set([t]))
            if M.rank(S) == target_rank:
                put_it_back = ground.difference(S)
                if all(M.rank(S.union(Set([t]))) == target_rank+1 for t in put_it_back):
                    bigflats.append(S)
            elif M.rank(S) == target_rank+1:
                work_stack.append(S)
            else: 
                print("My flat got too small too fast!")
    return bigflats
    




In [42]:
#This cell is the workhorse of the entire code. The parameter m decides how many boxes we will have in our first row, (m,1,1)


m = 7


w1, w2 = hook_words(m,2)

w2_words = list(Permutations(w2))

small, columnConvert = hook_specht_matrix(m,2)
smallt = small.transpose()
numcolS = len(small[0])

def makeSpecial(n): #This creates the table of all special flats
    special = {}
    
    for i in range(n):
        for j in range(i):
            special[(i,j)] = Set([Set([i,j,k]) for k in range(n) if k != i and k != j])
    return special

special = makeSpecial(m)
            

def nonzero_entries(): #This code creates a table of all the zero entries in each column of the sphect matrix. 
                      #This will allow us to later convert flats easily to their representations as sets of size 3.
    table = []
    num_rows = len(w2_words[0])
    for j in range(numcolS):
        table.append(Set([i for i in range(num_rows) if w2_words[columnConvert[j]][i] == 0]))
    return table

t = nonzero_entries()  

100%|██████████| 840/840 [00:00<00:00, 6495.45it/s]


In [40]:
def sensible_flat_name(F): #Converts a flat represented as a collection of columns in the specht matrix to a set of size 3
    return Set([t[i] for i in F])


def checkSpecial(n,F): #Checks if a given flat is a stable flat
    for (x,y) in special:
        if special[(x,y)].issubset(F):
            return (True,{x,y})
    return (False,{-1,0})

def createRandomSample(n,total): #Creates a collection of random subsets of [numCol] of size (m-1)(m-2)/2 - 1 with size total. 
                        #This will later decide which columns of the Specht matrix to span to construct codimension 1 flats.
    dim = (n-1)*(n-2)/2
    
    if total > binomial(numcolS,dim -1):
        total = binomial(numcolS,dim -1)
    
    ans = set()
    while len(ans) < total: 
        s = set()
        while len(s) < dim - 1:
            n = ZZ.random_element(0,numcolS)
            s.add(n)
        ans.add(Set(s))
    return ans

def extractLines(n,total): #Uses previous function to randomly construct subsets of [numcol], then looks at the associated 
                            #collections of columns of the specht matrix. Any such collection that spans a codimension 1 flat
                            #is recorded, while others are thrown away.
    dim = (n-1)*(n-2)/2
    S = createRandomSample(n,total)
    minLines = []
    for s in tqdm(S):
        vec = []
        for x in s:
            vec.append(smallt[x])
        minor = Matrix(vec)
        if minor.rank() == dim-1:
            minLines.append(s)
    return minLines

def convertToFlat(n,F): #inputs a collection of columns of the specht matrix, and expands it to a flat by finding all other
                        #columns which are linearly dependent on the original collection.
    dim = (n-1)*(n-2)/2
    
    tempF = set(F)
    
    remaining = [i for i in range(numcolS) if i not in tempF]
    
    for x in remaining:
        vec = [smallt[y] for y in tempF]
        vec.append(smallt[x])
        minor = Matrix(vec)
        if minor.rank() == dim-1:
            tempF.add(x)
    return Set(tempF)




In [47]:
#Looks at 300,000 randomly generated collections of columns, and checks each for the dimension of their span.
#Any collection with the correct dimensional span is recorded and converted to a flat.
lines = extractLines(m,300000)
A = set()
for F in tqdm(lines):
    A.add(convertToFlat(m,F))    

100%|██████████| 74788/74788 [08:50<00:00, 141.00it/s] 


In [49]:
#Takes the flats from the previous cell, and converts them to flats with the "right" name.
C = set()
for F in tqdm(A):
    C.add(sensible_flat_name(F))

100%|██████████| 4012/4012 [00:00<00:00, 46607.25it/s]


In [51]:
#print out the number of flats of each size found, along with the number of those flats that are stable and unstable

stablecount = dict()
unstablecount = dict()

for F in C:
    stablecount[len(F)] = 0
    unstablecount[len(F)] = 0
    
for F in C:
    if checkSpecial(m,F)[0]:
        stablecount[len(F)] += 1
    else:
        unstablecount[len(F)] +=1

for x in stablecount:
    print("size = {}, stable count = {}, unstable count = {}".format(x,stablecount[x],unstablecount[x]))
        
        
    

size = 23, stable count = 732, unstable count = 0
size = 22, stable count = 603, unstable count = 0
size = 20, stable count = 244, unstable count = 363
size = 18, stable count = 38, unstable count = 182
size = 24, stable count = 420, unstable count = 0
size = 16, stable count = 0, unstable count = 19
size = 19, stable count = 231, unstable count = 0
size = 21, stable count = 634, unstable count = 256
size = 26, stable count = 70, unstable count = 0
size = 27, stable count = 105, unstable count = 0
size = 17, stable count = 0, unstable count = 87
size = 30, stable count = 21, unstable count = 0
size = 15, stable count = 0, unstable count = 7


In [46]:
# randomly generate some lines in specht matroid, collated by flat size
from collections import defaultdict
def line_collate(m, trials):
    lines = extractLines(m,trials)
    C = set()
    for F in tqdm(lines):
        C.add(convertToFlat(m,F))

    flat_counts = defaultdict(set)
    for flat in C:
        flat_counts[len(flat)].add(flat)
    return flat_counts

#approximates the number of lines of each size by running the previous function twice, and looking at how much
#overlap occurs.

def approx_line_count(m, trials):
    print("Generating first samples and collating")
    tag = line_collate(m, trials)
    print("Generating second samples and collating")
    recapture = line_collate(m, trials)
    print("Approximate counting")

    valid_sizes = set(tag.keys()).union(set(recapture.keys()))
    for size in valid_sizes:
        tag_sample = tag[size]
        recapture_sample = recapture[size]
        overlap = tag_sample.intersection(recapture_sample)

        a,b,c = len(tag_sample), len(recapture_sample), len(overlap)
        if a==0 and b != 0:
            approx_count = str("b={} (no first sample)".format(b))
        elif b==0 and a != 0:
            approx_count = str("a={} (no second sample)".format(a))
        elif c==0:
            approx_count = str("unknown - no overlap between samples. a={},b={}".format(a,b))
        else:
            approx_count = str("{} by approximate counting. a = {}, b = {}, c = {}]".format(a*b/c,a,b,c))
        print("size {} - {}".format(size, approx_count))


approx_line_count(7, 5000000)

Generating first samples and collating


100%|██████████| 1248006/1248006 [5:54:51<00:00, 58.61it/s]    


Generating second samples and collating


100%|██████████| 1248514/1248514 [6:04:05<00:00, 57.15it/s]    


Approximate counting
size 14 - unknown - no overlap between samples. a=6,b=11
size 15 - 28026.0 by approximate counting. a = 162, b = 173, c = 1]
size 16 - 19172.222222222223 by approximate counting. a = 406, b = 425, c = 9]
size 17 - 12678.737967914438 by approximate counting. a = 1566, b = 1514, c = 187]
size 18 - 8486.638554216868 by approximate counting. a = 3195, b = 3307, c = 1245]
size 19 - 2527.9370078740158 by approximate counting. a = 2028, b = 2058, c = 1651]
size 20 - 2517.485294117647 by approximate counting. a = 2484, b = 2481, c = 2448]
size 21 - 1620.0 by approximate counting. a = 1620, b = 1620, c = 1620]
size 22 - 630.0 by approximate counting. a = 630, b = 630, c = 630]
size 23 - 735.0 by approximate counting. a = 735, b = 735, c = 735]
size 24 - 420.0 by approximate counting. a = 420, b = 420, c = 420]
size 26 - 70.0 by approximate counting. a = 70, b = 70, c = 70]
size 27 - 105.0 by approximate counting. a = 105, b = 105, c = 105]
size 30 - 21.0 by approximate coun